In [1]:
from data_source.vectorDatabaseChroma import create_database
%load_ext autoreload
%autoreload 2

In [2]:
chromadb,speakers_list_1,speakers_list_2,speakers_list_3,speakers_list_4,sec_form_names,earnings_call_quarter_vals = create_database("NVDA","2024",embed_type="sentence_transformer")

/home/athekunal/Finance Data/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/athekunal/Finance Data/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/athekunal/Finance Data/.venv/lib/python3.10/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 11060). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with y

Earnings Call Q1
Earnings Call Q2
Earnings Call Q3
Earnings Call Q4
SEC
Started Scraping
Scraped
Started Extracting
Extracted


In [3]:
chromadb,speakers_list_1,speakers_list_2,speakers_list_3,speakers_list_4,sec_form_names,earnings_call_quarter_vals = create_database("NVDA","2024",embed_type="sentence_transformer")

/home/athekunal/Finance Data/.venv/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [12]:
chromadb.similarity_search("What is the revenue breakdown of Nvidia?",k=5)

[Document(page_content='Thanks, Simona. Q3 was another record quarter. Revenue of $18.1 billion was up 34% sequentially and up more than 200% year-on-year and well above our outlook for $16 billion. Starting with Data Center. The continued ramp of the NVIDIA HGX platform based on our Hopper Tensor Core GPU architecture, along with InfiniBand end-to-end networking, drove record revenue of $14.5 billion, up 41% sequentially and up 279% year-on-year. NVIDIA HGX with InfiniBand together are essentially the reference architecture for', metadata={'quarter': 'Q3', 'speaker': 'Colette Kress'}),
 Document(page_content="financial results for the second quarter of fiscal 2024. The content of today's call is NVIDIA's property. It can't be reproduced or transcribed without our prior written consent. During this call, we may make forward-looking statements based on current expectations. These are subject to a number of significant risks and uncertainties and our actual results may differ materially.

In [19]:
chromadb.similarity_search("What is the revenue breakdown of Nvidia?",k=5,filter={
        "filing_type":{"$eq":"10-Q2"}
    })

[Document(page_content='Financial Statements included in our Annual Report on Form 10-K for the fiscal year ended January\xa028, 2024. We operate on a 52- or 53-week year, ending on the last Sunday in January. Fiscal years 2025 and 2024 are both 52-week years. The first quarters of fiscal years 2025 and 2024 were both 13-week quarters. Our condensed consolidated financial statements include the accounts of NVIDIA Corporation and our wholly-owned subsidiaries. All intercompany balances and transactions have been eliminated in', metadata={'accessionNumber': '000104581024000124', 'filingDate': '2024-05-29', 'filing_type': '10-Q2', 'reportDate': '2024-04-28', 'sectionName': 'FINANCIAL_STATEMENTS'}),
 Document(page_content='in privately negotiated transactions, pursuant to a Rule 10b5-1 trading plan or in structured share repurchase programs, and can be made in one or more larger repurchases, in compliance with Rule 10b-18 of the Exchange Act, subject to market conditions, applicable legal 

In [16]:
q = chromadb.get(
    where={
        "filing_type":{"$eq":"10-Q2"}
    }
)

In [18]:
q['metadatas'][0]

{'accessionNumber': '000104581024000124',
 'filingDate': '2024-05-29',
 'filing_type': '10-Q2',
 'reportDate': '2024-04-28',
 'sectionName': 'RISK_FACTORS'}

In [20]:
quarter_speaker_dict = {
    "Q1":speakers_list_1,
    "Q2":speakers_list_2,
    "Q3":speakers_list_3,
    "Q4":speakers_list_4
}

In [27]:
from typing import Annotated
def query_database_earnings_call(
        question = Annotated[str,"Question to query the database for relevant documents"],
        quarter = Annotated[str,"the financial quarter that is discussed in the question and possible options are Q1, Q2, Q3, Q4"],
):
    assert quarter in earnings_call_quarter_vals, "The quarter should be from Q1, Q2, Q3, Q4"
    
    req_speaker_list = []
    quarter_speaker_list = quarter_speaker_dict[quarter]

    for sl in quarter_speaker_list:
        if sl in question or sl.lower() in question:
            req_speaker_list.append(sl)
    if len(req_speaker_list) == 0:
        req_speaker_list = quarter_speaker_list
    
    relevant_docs = chromadb.similarity_search(
        question,
        k=5,
        filter={
            "$and":[
                {
                    "quarter":{"$eq":quarter}
                },
                {
                    "speaker":{"$in":req_speaker_list}
                }
            ]
        }
    )

    speaker_releavnt_dict = {}
    for doc in relevant_docs:
        speaker = doc.metadata['speaker']
        speaker_text = doc.page_content
        if speaker not in speaker_releavnt_dict:
            speaker_releavnt_dict[speaker] = speaker_text
        else:
            speaker_releavnt_dict[speaker] += " "+speaker_text
    
    relevant_speaker_text = ""
    for speaker, text in speaker_releavnt_dict.items():
        relevant_speaker_text += speaker + ": "
        relevant_speaker_text += text + "\n\n"

    return relevant_speaker_text


In [31]:
val = query_database_earnings_call(
    "Revenue breakdown",
    "Q4"
)

In [32]:
print(val)

Colette Kress: we returned $2.8 billion to shareholders in the form of share repurchases and cash dividends. During fiscal year '24, we utilized cash of $9.9 billion towards shareholder returns, including $9.5 billion in share repurchases. Let me turn to the outlook for the first quarter. Total revenue is expected to be $24 billion, plus or minus 2%. We expect sequential growth in data center and proviz, partially offset by seasonal decline in gaming. GAAP and non-GAAP gross margins are expected to be 76.3% and 77% Thanks, Simona. Q4 was another record quarter. Revenue of $22.1 billion was up 22% sequentially and up to 265% year-on-year and well above our outlook of $20 billion. For fiscal 2024, revenue was $60.9 billion and up 126% from the prior year. Starting with data center. Data center revenue for the fiscal 2024 year was $47.5 billion, more than tripling from the prior year. The world has reached the tipping point of new computing era. The $1 trillion installed base of data cent

In [33]:
def query_database_sec(
        question = Annotated[str,"Question to query the database for relevant documents"],
        sec_form_name = Annotated[str,"SEC FORM NAME that the question is talking about. It can be 10-K for yearly data and 10-Q for quarterly data. For quarterly data, it can be 10-Q2 to represent Quarter 2 and similarly for other quarters"]
):
    assert sec_form_name in sec_form_names, f'The search form type should be in {sec_form_names}'
    
    relevant_docs = chromadb.similarity_search(
        question,
        k=5,
        filter={
            "filing_type":{"$eq":sec_form_name}
        }
    )

    relevant_section_dict = {}
    for doc in relevant_docs:
        section = doc.metadata['sectionName']
        section_text = doc.page_content
        if section not in relevant_section_dict:
            relevant_section_dict[section] = section_text
        else:
            relevant_section_dict[section] += " "+section_text
    
    relevant_section_text = ""
    for section, text in relevant_section_dict.items():
        relevant_section_text += section + ": "
        relevant_section_text += text + "\n\n"

    return relevant_section_text

In [34]:
query_database_sec(
    "What is the revenue breakdown of Nvidia?",
    "10-K"
)

"EXHIBITS: Consolidated Statements of Income for the years ended January 28, 2024, January 29, 2023, and January 30, 2022 The exhibits listed in the accompanying index to exhibits are filed or incorporated by reference as a part of this Annual Report on Form 10-K. We have audited the accompanying consolidated balance sheets of NVIDIA Corporation and its subsidiaries (the “Company”) as of January 28, 2024 and January 29, 2023, and the related consolidated statements of income, comprehensive income, shareholders' equity in April 1993 and reincorporated in Delaware in April 1998. All references to “NVIDIA,” “we,” “us,” “our” or the “Company” mean NVIDIA Corporation and its subsidiaries. We operate on a 52- or 53-week year, ending on the last Sunday in January. Fiscal years 2024, 2023 and 2022 were all 52-week years. Our consolidated financial statements include the accounts of NVIDIA Corporation and our wholly-owned subsidiaries. All intercompany balances and transactions have been elimin